<a href="https://colab.research.google.com/github/arghavan-b/QLoRA/blob/main/Llama_3_1_8b_fine_tuning_LoRa%2BUnsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Load the pre-trained model and tokenizer using the FastLanguageModel class

In [2]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
max_seq_length = 2048
dtype = None # Set to None for auto-detection based on the hardware
load_in_4bit = True # Setting this to True significantly reduces VRAM consumption, Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Apply Low-Rank Adaptation (LoRA) to the model using Unsloth's get_peft_model function

In [3]:
peft_model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # Scaling factor for the LoRA updates; often set equal to the rank 'r'
    lora_dropout = 0,
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # Enables Unsloth's optimized gradient checkpointing to save memory
    random_state = 3407,
    use_rslora = False,  # Whether to use Rank Stabilized LoRA
    loftq_config = None, # Configuration for Low-bit Fine-Tuning Quantization (LoftQ); set to None if not using
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
def number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(number_of_trainable_model_parameters(peft_model))

trainable model parameters: 41943040
all model parameters: 4582543360
percentage of trainable model parameters: 0.92%


##Data Prep

In [4]:
dialog_dataset = load_dataset("knkarthick/dialogsum")

In [9]:
dialog_dataset['train'][10]

{'id': 'train_10',
 'dialogue': "#Person1#: Could you do me a favor?\n#Person2#: Sure. What is it?\n#Person1#: Could you run over to the store? We need a few things.\n#Person2#: All right. What do you want me to get?\n#Person1#: Well, could you pick up some sugar?\n#Person2#: Okay. How much?\n#Person1#: A small bag. I guess we also need a few oranges.\n#Person2#: How many?\n#Person1#: Oh, let's see. . . About six.\n#Person2#: Anything else?\n#Person1#: Yes. We're out of milk.\n#Person2#: Okay. How much do you want me to get? A gallon?\n#Person1#: No. I think a half gallon will be enough.\n#Person2#: Is that all?\n#Person1#: I think so. Have you got all that?\n#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.\n#Person1#: Do you have enough money?\n#Person2#: I think so.\n#Person1#: Thanks very much. I appreciate it.",
 'summary': '#Person1# asks #Person2# to do a favor. #Person2# agrees and helps buy a small bag of sugar, six oranges, and a half-gallon 

In [5]:
dialog_prompt = """Below is a conversation, your task is to summarize the conversation.

### Conversation:
{}

### Summary:
{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    dialogues = examples["dialogue"]
    summaries = examples["summary"]
    texts = []
    for dialogue, summary in zip(dialogues, summaries):
        text = dialog_prompt.format(dialogue, summary) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
train_dataset = dialog_dataset['train'].map(formatting_prompts_func, batched = True,)

In [6]:
dialog_dataset.shape

{'train': (12460, 4), 'validation': (500, 4), 'test': (1500, 4)}

In [7]:
import gc
# Clear cache and collect garbage
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

## Train the model

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = peft_model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        save_steps = 10,
        logging_steps = 10,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/12460 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
peft_model.config.use_cache = False
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 12,460 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.371900
20,1.318600
30,1.327400
40,1.308500
50,1.338000
60,1.375800
70,1.343300
80,1.375400
90,1.373100
100,1.338200


##Inference

In [11]:
dialog_dataset['test'][0]

{'id': 'test_0_1',
 'dialogue': "#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much tim

In [14]:

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    dialog_prompt.format(
        dialog_dataset['test'][0]['dialogue'], # prompt
        ""# response - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is a conversation, your task is to summarize the conversation.\n\n### Conversation:\n#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

This ONLY saves the LoRA adapters, and not the full model.

In [ ]:
model.save_pretrained('peft_model')
tokenizer.save_pretrained('peft_model')

In [ ]:
from unsloth import FastLanguageModel

dialog_prompt = """Below is a conversation, your task is to summarize the conversation.

### Conversation:
{}

### Summary:
{}"""
max_seq_length = 2048
dtype = None # Set to None for auto-detection based on the hardware
load_in_4bit = True # Setting this to True significantly reduces VRAM consumption, Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "peft_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
         device_map="auto",
    offload_folder="offload",  # Specify a folder for offloaded
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    dialog_prompt.format(

        dialog_dataset['test'][1]['dialogue'], # prompt
        "", # response - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [1]:
model.save_pretrained_merged("llama3.1-8B-lora-summarize-dialog", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("abahadori/llama3.1-8B-lora-summarize-dialog", tokenizer, save_method = "merged_16bit", token = "")

NameError: name 'model' is not defined